In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
#크롤링 시작
result = pd.DataFrame()
for i in range(603600,603696):
    URL = "http://www1.president.go.kr/petitions/"+str(i)

    response = requests.get(URL)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span', class_='counter')

    for content in soup.select('div.petitionsView_write > div.View_write'): content
    a=[]
    for tag in soup.select('ul.petitionsView_info_list > li'):
        a.append(tag.contents[1])

    if len(a) != 0:
        df1=pd.DataFrame({'start':[a[1]],
                          'end':[a[2]],
                          'category': [a[0]],
                          'count':[count.text],
                          'title': [title.next],
                          'content': [content.text.strip()[0:13000]]
                          })
        
        result= pd.concat([result,df1])
        result.index = np.arange(len(result))

    if i% 60 == 0:
        print("Sleep 10sec. Count:" + str(i)
            +", Local time:"
            + time.strftime('%Y-%m-%d', time.localtime(time.time()))
            +" "+time.strftime('%X', time.localtime(time.time()))
            +", Data Length:"+ str(len(result)))
        time.sleep(10)


Sleep 10sec. Count:603600, Local time:2022-01-12 05:33:44, Data Length:1
Sleep 10sec. Count:603660, Local time:2022-01-12 05:35:20, Data Length:59


In [ ]:
#크롤링 데이터 확인
print(result.shape)

df=result
df.head()

(59, 6)


,start,end,category,count,title,content
0,2022-01-06,2022-02-05,보건복지,926,코로나 백신 접종 후 부작용을 겪고 있는 사람들에게 관심을 가져주세요,안녕하세요 인천에서 살고 있는 30대 직장인입니다\r\n지금 이순간에도 백신접종 후...
1,2022-01-06,2022-02-05,보건복지,"1,232",도서관에 대한 백신 패스를 당장 중단하십시오,미접종 대학원생입니다. 백신패스 제도 시행으로 인해 공공도서관뿐 아니라 사립대학의 ...
2,2022-01-06,2022-02-05,기타,213,반드시 근절해 주세요!,안녕하세요? 농협중앙회 상호금융소비자보호부 ******팀장 *** 입니다.\r\n저...
3,2022-01-06,2022-02-05,보건복지,"4,292",부스터샷 20일후 급성혈액암,"저는 만63세로 부산에서 작은 서점을 하고 있습니다 1, 2차 백신접종은 진통제..."
4,2022-01-06,2022-02-05,교통/건축/국토,272,가로주택정비사업(빈집및소규모주택정비사업에관한 특례법)의 도로 신설시 도로면적 사업시...,좁은 부지와 세대수가 많은 공동주택의 경우 아파트가 30~40년 이상 되어도 주위 ...


In [ ]:
df.loc[1]['content']

'미접종 대학원생입니다. 백신패스 제도 시행으로 인해 공공도서관뿐 아니라 사립대학의 도서관까지 출입이 제한되었습니다. 누구보다 정부가 권고한 방역 및 위생수칙을 잘 지키며 긴 시간을 참아왔는데 돌아온 것은 당장 학업에 필요한 교재와 논문들에까지 접근이 제한된 상황뿐입니다. 열람실 이용도 아닌 출입과 대출마저 막힌 상황은 학업 권리를 심각하게 침해하고 있습니다. 식당에서 밥도 먹을 수 있고 종교시설에서 예배도 드릴 수 있는데, 마스크 쓰고 들어가서 책만 빌려 나오는 도서관에 제가 들어가지 못할 합리적 근거는 어떤 곳에서도 찾을 수 없습니다. 공공도서관 및 대학 도서관에 대한 방역패스 조치를 중단하십시오. 기타 문화생활에 대한 제약은 불편의 범위로 여기며 참아왔지만, 학업에 대한 권리가 부당하게 침해받는 상황은 직업과 생활의 문제로 연결되고 있습니다. 독서실과 학원에 대해서도 시행이 중단된 상황에서 학교 도서관을 출입할 수 없다는 것은 정말 어불성설이며 정책의 일관성과 형평성도 상실한 조치입니다. 건강을 보호한다는 이익보다 기본권의 침해가 심각한 상황입니다. 부디 신속한 조치를 취해 주시길 부탁드립니다. 민주주의 헌법 위에 국민의 손으로 세워진 정부를 자랑스럽게 여기는 국민이고 싶습니다.'

In [ ]:
# 전처리
import re
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('^ ㄱ-ㅣ가-힣 0-9+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [ ]:
df.loc[1]['content']

'미접종 대학원생입니다. 백신패스 제도 시행으로 인해 공공도서관뿐 아니라 사립대학의 도서관까지 출입이 제한되었습니다. 누구보다 정부가 권고한 방역 및 위생수칙을 잘 지키며 긴 시간을 참아왔는데 돌아온 것은 당장 학업에 필요한 교재와 논문들에까지 접근이 제한된 상황뿐입니다. 열람실 이용도 아닌 출입과 대출마저 막힌 상황은 학업 권리를 심각하게 침해하고 있습니다. 식당에서 밥도 먹을 수 있고 종교시설에서 예배도 드릴 수 있는데, 마스크 쓰고 들어가서 책만 빌려 나오는 도서관에 제가 들어가지 못할 합리적 근거는 어떤 곳에서도 찾을 수 없습니다. 공공도서관 및 대학 도서관에 대한 방역패스 조치를 중단하십시오. 기타 문화생활에 대한 제약은 불편의 범위로 여기며 참아왔지만, 학업에 대한 권리가 부당하게 침해받는 상황은 직업과 생활의 문제로 연결되고 있습니다. 독서실과 학원에 대해서도 시행이 중단된 상황에서 학교 도서관을 출입할 수 없다는 것은 정말 어불성설이며 정책의 일관성과 형평성도 상실한 조치입니다. 건강을 보호한다는 이익보다 기본권의 침해가 심각한 상황입니다. 부디 신속한 조치를 취해 주시길 부탁드립니다. 민주주의 헌법 위에 국민의 손으로 세워진 정부를 자랑스럽게 여기는 국민이고 싶습니다.'

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 6.3 MB/s 
     |████████████████████████████████| 448 kB 61.1 MB/s 


In [ ]:
#토크나이징 
from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

In [ ]:
#파생변수 생성
df['token_final'] = df.title_token + df.content_token
df['count']=df['count'].replace({',' : ''}, regex=True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
dtype: object


In [ ]:
df_drop = df[['token_final','label']]

print(df_drop)
df_drop.head()

                                          token_final label
0   [코로나, 백신, 접종, 후, 부작용, 을, 겪고, 있는, 사람, 들, 에게, 관심...    No
1   [도서관, 에, 대한, 백신, 패스, 를, 당장, 중단, 하십시오, 미접, 종, 대...   Yes
2   [반드시, 근절, 해, 주세요, !, 농협, 중앙회, 상호금융, 소비자보호, 부, ...    No
3   [부스터, 샷, 20일, 후, 급성, 혈액암, 저, 만, 세로, 부산, 서점, 차,...   Yes
4   [가로, 주택, 정비, 사업, (, 빈집, 및, 소규모, 주택, 정비, 사업, 에,...    No
5   [백신, 패스, 도입, 에, 관련, 된, 모든, 공무원, 들, 을, 즉각, 파면, ...   Yes
6   [울산, 울주군, 청량읍, 초등학교, 및, 고등학교, 신설, 촉구, 합니다, ., ...    No
7   [요, 양, 병원, 에, 계신, 엄마, 를, 만나게, 해주세요, 저, 어머니, 뇌졸...    No
8   [의문, 투성이, 의, **, 경찰서, 불, 송치, 결정, 사건번호, 형제, 사기,...    No
9   [학원, 강사, 지만, 학원, 문, 좀, 닫았으면, 좋겠어요, 강남, 학원, 강사,...    No
10  [허리협착증, 으로, 인한, 시술, ,, 수술, 로, 더, 악화시킨, 병원, 을, ...    No
11  [미접, 종자, 를, 사회, 와, 단절, 시키는, 백신, 패스, 확대, 반대, 와,...   Yes
12  [백신, 맞는, 여성, 이, 라면, 꼭, 읽어주세요, 혹시, 백신, 부정, 출혈, ...   Yes
13  [임산부, 및, 영유, 아, 자녀, 를, 둔, 엄마, 의, 백, 신, 패스, 를, ...   Yes
14  [****, 반려, 회사, 폰, 지, 사기피해, 4만, 명, 회사, 알, 계기, 사...    No
15  [억울한, 피해자, 를, 검사, 님, ,, 판사, 님, 도와주세요

,token_final,label
0,"[코로나, 백신, 접종, 후, 부작용, 을, 겪고, 있는, 사람, 들, 에게, 관심...",No
1,"[도서관, 에, 대한, 백신, 패스, 를, 당장, 중단, 하십시오, 미접, 종, 대...",Yes
2,"[반드시, 근절, 해, 주세요, !, 농협, 중앙회, 상호금융, 소비자보호, 부, ...",No
3,"[부스터, 샷, 20일, 후, 급성, 혈액암, 저, 만, 세로, 부산, 서점, 차,...",Yes
4,"[가로, 주택, 정비, 사업, (, 빈집, 및, 소규모, 주택, 정비, 사업, 에,...",No


In [ ]:
#단어 임베딩
from gensim.models import Word2Vec
embedding_model = Word2Vec(df_drop['token_final'], sg = 1, size = 100, window = 2, min_count = 1, workers = 4)
# sg 0은 cbow, 1은 skip gram임. workers 는 병렬 처리 프로세서 수 보통 4~6 지정

print(embedding_model)

Word2Vec(vocab=3593, size=100, alpha=0.025)


In [ ]:
# 임의의 단어 넣어서 확인
model_result = embedding_model.wv.most_similar("코로나")
print(model_result)

[('병원', 0.9977469444274902), ('것', 0.9975653886795044), ('그', 0.997255802154541), ('할머니', 0.9972310066223145), ('수', 0.9972198605537415), ('이', 0.9970903396606445), ('검사', 0.9969830513000488), ('택배', 0.9969385862350464), ('위해', 0.9969356060028076), ('저', 0.9969030618667603)]


In [ ]:
#임베딩 모델 저장 및 로드
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('/content/petitions_tokens_w2v')
loaded_model = KeyedVectors.load_word2vec_format("/content/petitions_tokens_w2v")

model_result = loaded_model.most_similar("코로나")
print(model_result)

[('병원', 0.9977469444274902), ('것', 0.9975653886795044), ('그', 0.997255802154541), ('할머니', 0.9972310066223145), ('수', 0.9972198605537415), ('이', 0.9970903396606445), ('검사', 0.9969830513000488), ('택배', 0.9969385862350464), ('위해', 0.9969356060028076), ('저', 0.9969030618667603)]


In [ ]:
#데이터셋 분할 및 저장
from numpy.random import RandomState
rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('/content/train.csv', index=False, encoding = 'utf-8-sig')
val.to_csv('/content/validation.csv', index=False, encoding='utf-8-sig')

In [ ]:
pip install torchtext

In [ ]:
#Field 클래스 정의

import torchtext
from torchtext.legacy.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']','',str(text))
    text = text.split(',')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

In [ ]:
# 데이터 불러오기
from torchtext.legacy.data import TabularDataset

train, validation = TabularDataset.splits(path = '/content/', train = 'train.csv', validation= 'validation.csv' , format = 'csv', fields = [('text', TEXT), ('label', LABEL)], skip_header=True)

print("Train:", train[0].text, train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['백신', ' 패스', ' 예외', ' 자', ' 기준', ' 을', ' 개선', ' 해', ' 주세요', ' .', ' 전', ' 형', ' 간염', ' 주사', ' 길', ' 바레', ' 증후군', ' 아들', ' 독감', ' 주사', ' 스티븐', ' 존슨', ' 증후군', ' 대학병원', ' 입원', ' 한적', ' 아들', ' 퇴원', ' 때', ' 의사', ' 선생님', ' 혹시', ' 평생', ' 독감', ' 백신', ' 말', ' 그', ' 이후', ' 가족', ' 독감', ' 백신', ' 적', ' 코로나', ' 백신', ' 부작용', ' 길', ' 바레', ' 증후군', ' 스티븐', ' 존슨', ' 증후군', ' 현재', ' 저', ' 애', ' 백신', ' 못', ' 패스', ' 도입', ' 후', ' 저', ' 카페', ' 음식점', ' 거의', ' 집', ' 문제', ' 항생제', ' 부작용', ' 콜린', ' 두드러기', ' 중학생', ' 아들', ' 걱정', ' 예전', ' 입원', ' 대학병원', ' 예외', ' 진단', ' 좀', ' 달라', ' 의사', ' 선생님', ' 부탁', ' 이유', ' 아스', ' 트', ' 카', ' 얀', ' 길', ' 바레', ' 증후군', ' 스티븐', ' 존슨', ' 증후군', ' 부작용', ' 화이자', ' 더나', ' 백신', ' 부작용', ' 후기', ' 길', ' 바레', ' 증후군', ' 스티븐', ' 존슨', ' 증후군', ' 학회', ' 보고', ' 부작용', ' 때', ' 고생', ' 생각', ' 도저히', ' 용기', ' 저', ' 해도', ' 중학교', ' 학년', ' 아들', ' 치사', ' 율도', ' 부작용', ' 또', ' 청소년', ' 패스', ' 도입', ' 백신', ' 정말', ' 이건', ' 생각', ' 듭니', ' 제발', ' 다른', ' 백신', ' 부작용', ' 백신', ' 패스', ' 예외', ' 지정', ' 수', ' 기준', ' 개선'

In [ ]:
#단어장 및 DataLoader 정의
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors = Vectors(name="/content/petitions_tokens_w2v")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)

LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(datasets = (train, validation), batch_size=8, device = device, sort = False)

print('임베딩 벡터의 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

100%|██████████| 3593/3593 [00:00<00:00, 17298.93it/s]


임베딩 벡터의 개수와 차원 : torch.Size([3261, 100])


In [ ]:
#TextCNN 모델링
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):

    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):

        super(TextCNN, self).__init__()

        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)

    def forward(self,x):

        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]

        fc_x = torch.cat(pool_x, dim=1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.dropout(fc_x)

        logit = self.fc(fc_x)

        return logit

In [ ]:
#모델 학습 함수 정의

def train(model, device, train_itr, optimizer):

    model.train()
    corrects, train_loss = 0.0,0

    for batch in train_itr:

        text, target = batch.text, batch.label
        text = torch.transpose(text,0,1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        optimizer.zero_grad()
        logit = model(text)

        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

In [ ]:
#모델 평가 함수 정의
def evaluate(model, device, itr):

    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:

        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)

        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects/len(itr.dataset)

    return test_loss, accuracy

In [ ]:
#모델 학습 및 성능 확인

model = TextCNN(vocab, 100, 10, [3,4,5], 2).to(device)

print(model)

TextCNN(
  (embed): Embedding(3261, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)

    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))

    val_loss, val_acc = evaluate(model, device, validation_iter)

    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))

    if val_acc > best_test_acc:
        best_test_acc = val_acc

        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")

    print('------------------------------------------------')

Train Epoch: 1 	 Loss: 0.08923530324976495 	 Accuracy: 44.680850982666016%
Valid Epoch: 1 	 Loss: 0.11951988438765208 	 Accuracy: 33.333335876464844%
model saves at 33.333335876464844 accuracy
------------------------------------------------
Train Epoch: 2 	 Loss: 0.08833931481584589 	 Accuracy: 44.680850982666016%
Valid Epoch: 2 	 Loss: 0.11771067976951599 	 Accuracy: 33.333335876464844%
------------------------------------------------
Train Epoch: 3 	 Loss: 0.08803323989218854 	 Accuracy: 57.446807861328125%
Valid Epoch: 3 	 Loss: 0.11637566486994426 	 Accuracy: 33.333335876464844%
------------------------------------------------
